<a href="https://colab.research.google.com/github/apache/beam/blob/master/examples/notebooks/healthcare/beam_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License

# **Natural Language Processing Pipeline**

**Note**: This example is used from [here](https://github.com/rasalt/healthcarenlp/blob/main/nlp_public.ipynb).



This example demonstrates how to set up an Apache Beam pipeline that reads a file from [Google Cloud Storage](https://https://cloud.google.com/storage), and calls the [Google Cloud Healthcare NLP API](https://cloud.google.com/healthcare-api/docs/how-tos/nlp) to extract information from unstructured data. This application can be used in contexts such as reading scanned clinical documents and extracting structure from it.

An Apache Beam pipeline is a pipeline that reads input data, transforms that data, and writes output data. It consists of PTransforms and PCollections. A PCollection represents a distributed data set that your Beam pipeline operates on. A PTransform represents a data processing operation, or a step, in your pipeline. It takes one or more PCollections as input, performs a processing function that you provide on the elements of that PCollection, and produces zero or more output PCollection objects.

For details about Apache Beam pipelines, including PTransforms and PCollections, visit the [Beam Programming Guide](https://beam.apache.org/documentation/programming-guide/).

You'll be able to use this notebook to explore the data in each PCollection.

First, lets install the necessary packages.

In [ ]:
!pip install apache-beam[gcp]

 **GCP Setup**

1. Authenticate your notebook by `gcloud auth application-default login` in the Colab terminal.

2. Run `gcloud config set project <YOUR-PROJECT>`

Set the variables in the next cell based upon your project and preferences. The files referred to in this notebook nlpsample*.csv are in the format with one
blurb of clinical note.

Note that below, **us-central1** is hardcoded as the location. This is because of the limited number of [locations](https://cloud.google.com/healthcare-api/docs/how-tos/nlp) the API currently supports.

In [ ]:
DATASET="<YOUR-BQ-DATASEST>" # @param {type:'string'}
TEMP_LOCATION="<YOUR-TEMP-LOCATION>" # @param {type:'string'}
PROJECT='<YOUR-PROJECT>'# @param {type:'string'}
LOCATION='us-central1'
URL=f'https://healthcare.googleapis.com/v1/projects/{PROJECT}/locations/{LOCATION}/services/nlp:analyzeEntities'
NLP_SERVICE=f'projects/{PROJECT}/locations/{LOCATION}/services/nlp'

Then, download [this raw CSV file](https://github.com/socd06/medical-nlp/blob/master/data/test.csv), and then upload it into Colab. You should be able to view this file (*test.csv*) in the "Files" tab in Colab after uploading.

**BigQuery Setup**

We will be using BigQuery to warehouse the structured data revealed in the output of the Healthcare NLP API. For this purpose, we create 3 tables to organize the data. Specifically, these will be table entities, table relations, and table entity mentions, which are all outputs of interest from the Healthcare NLP API.

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.

TABLE_ENTITY="entity"


schemaEntity = [
    bigquery.SchemaField("entityId", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("preferredTerm", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("vocabularyCodes", "STRING", mode="REPEATED"),
]


client = bigquery.Client()

# Create Table IDs
table_ent = PROJECT+"."+DATASET+"."+TABLE_ENTITY


# If table exists, delete the tables.
client.delete_table(table_ent, not_found_ok=True)


# Create tables

table = bigquery.Table(table_ent, schema=schemaEntity)
table = client.create_table(table)  # Make an API request.

print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.

TABLE_REL="relations"

schemaRelations = [
    bigquery.SchemaField("subjectId", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("objectId", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("confidence", "FLOAT64", mode="NULLABLE"),
    bigquery.SchemaField("id", "STRING", mode="NULLABLE"),
]

client = bigquery.Client()

# Create Table IDs

table_rel = PROJECT+"."+DATASET+"."+TABLE_REL

# If table exists, delete the tables.

client.delete_table(table_rel, not_found_ok=True)

# Create tables

table = bigquery.Table(table_rel, schema=schemaRelations)
table = client.create_table(table)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)




In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.

TABLE_ENTITYMENTIONS="entitymentions"

schemaEntityMentions = [
    bigquery.SchemaField("mentionId", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField(
        "text",
        "RECORD",
         mode="NULLABLE",
         fields=[
             bigquery.SchemaField("content", "STRING", mode="NULLABLE"),
             bigquery.SchemaField("beginOffset", "INTEGER", mode="NULLABLE"),
         ],
    ),
    bigquery.SchemaField(
        "linkedEntities",
        "RECORD",
         mode="REPEATED",
         fields=[
             bigquery.SchemaField("entityId", "STRING", mode="NULLABLE"),
         ],
    ),
    bigquery.SchemaField(
        "temporalAssessment",
        "RECORD",
         mode="NULLABLE",
         fields=[
             bigquery.SchemaField("value", "STRING", mode="NULLABLE"),
             bigquery.SchemaField("confidence", "FLOAT64", mode="NULLABLE"),
         ],
    ),
    bigquery.SchemaField(
        "certaintyAssessment",
        "RECORD",
         mode="NULLABLE",
         fields=[
             bigquery.SchemaField("value", "STRING", mode="NULLABLE"),
             bigquery.SchemaField("confidence", "FLOAT64", mode="NULLABLE"),
         ],
    ),
    bigquery.SchemaField(
        "subject",
        "RECORD",
         mode="NULLABLE",
         fields=[
             bigquery.SchemaField("value", "STRING", mode="NULLABLE"),
             bigquery.SchemaField("confidence", "FLOAT64", mode="NULLABLE"),
         ],
    ),
    bigquery.SchemaField("confidence", "FLOAT64", mode="NULLABLE"),
    bigquery.SchemaField("id", "STRING", mode="NULLABLE")
]

client = bigquery.Client()

# Create Table IDs

table_mentions = PROJECT+"."+DATASET+"."+TABLE_ENTITYMENTIONS

# If table exists, delete the tables.

client.delete_table(table_mentions, not_found_ok=True)

# Create tables

table = bigquery.Table(table_mentions, schema=schemaEntityMentions)
table = client.create_table(table)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

**Pipeline Setup**

We will use InteractiveRunner in this notebook.

In [ ]:
# Python's regular expression library
import re
from sys import argv
# Beam and interactive Beam imports
import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

#Reference https://cloud.google.com/dataflow/docs/guides/specifying-exec-params#python_1
from apache_beam.options.pipeline_options import PipelineOptions

runnertype = "InteractiveRunner"

options = PipelineOptions(
    flags=argv,
    runner=runnertype,
    project=PROJECT,
    job_name="my-healthcare-nlp-job",
    temp_location=TEMP_LOCATION,
    region=LOCATION)

The following defines a `PTransform` named `ReadLinesFromText`, that extracts lines from a file.

In [ ]:
class ReadLinesFromText(beam.PTransform):

    def __init__(self, file_pattern):
        self._file_pattern = file_pattern

    def expand(self, pcoll):
        return (pcoll.pipeline
                | beam.io.ReadFromText(self._file_pattern))

The following sets up an Apache Beam pipeline with the *Interactive Runner*. The *Interactive Runner* is the runner suitable for running in notebooks. A runner is an execution engine for Apache Beam pipelines.

In [ ]:
p = beam.Pipeline(options = options)

The following sets up a PTransform that extracts words from a Google Cloud Storage file that contains lines with each line containing a In our example, each line is a medical notes excerpt that will be passed through the Healthcare NLP API

**"|"** is an overloaded operator that applies a PTransform to a PCollection to produce a new PCollection. Together with |, >> allows you to optionally name a PTransform.

Usage:[PCollection] | [PTransform], **or** [PCollection] | [name] >> [PTransform]

In [ ]:
lines = p | 'read' >> ReadLinesFromText("test.csv")

We then write a **DoFn** that will invoke the [NLP API](https://cloud.google.com/healthcare-api/docs/how-tos/nlp).

In [ ]:
class InvokeNLP(beam.DoFn):

    def process(self, element):
      #  import requests
        import uuid
        from google.auth import compute_engine
        credentials = compute_engine.Credentials()
        from google.auth.transport.requests import AuthorizedSession
        authed_session = AuthorizedSession(credentials)
        url = URL
        payload = {
            'nlp_service': NLP_SERVICE,
            'document_content': element
        }
        resp = authed_session.post(url, data=payload)
        response = resp.json()
        response['id'] = uuid.uuid4().hex[:8]
        yield response

class AnalyzeLines(beam.PTransform):
    def expand(self, pcoll):
        return (
            pcoll
            | "Invoke NLP API" >> beam.ParDo(InvokeNLP())
        )

From our elements, being processed, we will get the entity mentions, relationships, and entities respectively.

In [ ]:
import json
from apache_beam import pvalue

class breakUpEntities(beam.DoFn):
    def process(self, element):
        for e in element['entities']:
            print(e)
            yield e

class getRelationships(beam.DoFn):
    def process(self, element):
        obj = {}
        id = element['id']
        for e in element['relationships']:
            obj = e
            obj['id'] = id
            yield obj

class getEntityMentions(beam.DoFn):
    def process(self, element):
        obj = {}
        for e in element['entityMentions']:
            e['id'] = element['id']
            yield e


In [ ]:
from apache_beam.io.gcp.internal.clients import bigquery


table_spec = bigquery.TableReference(
    projectId=PROJECT,
    datasetId=DATASET,
    tableId=TABLE_ENTITY)

nlp_annotations = (lines
                | "Analyze" >> AnalyzeLines()
                  )


We then write these results to [BigQuery](https://cloud.google.com/bigquery), a cloud data warehouse.

In [ ]:
resultsEntities = ( nlp_annotations
                | "Break" >> beam.ParDo(breakUpEntities())
                | "WriteToBigQuery" >> beam.io.WriteToBigQuery(
                    table_spec,
                    write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND,
                    create_disposition=beam.io.BigQueryDisposition.CREATE_NEVER)
                  )

In [ ]:
table_spec = bigquery.TableReference(
    projectId=PROJECT,
    datasetId=DATASET,
    tableId=TABLE_REL)

resultsRelationships = ( nlp_annotations
                | "GetRelationships" >>  beam.ParDo(getRelationships())
                | "WriteToBigQuery" >> beam.io.WriteToBigQuery(
                    table_spec,
                    write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND,
                    create_disposition=beam.io.BigQueryDisposition.CREATE_NEVER)
                  )

In [ ]:
table_spec = bigquery.TableReference(
    projectId=PROJECT,
    datasetId=DATASET,
    tableId=TABLE_ENTITYMENTIONS)

resultsEntityMentions = ( nlp_annotations
                | "GetEntityMentions" >> beam.ParDo(getEntityMentions())
                | "WriteToBigQuery" >> beam.io.WriteToBigQuery(
                    table_spec,
                    write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND,
                    create_disposition=beam.io.BigQueryDisposition.CREATE_NEVER)
                  )

You can see the job graph for the pipeline by doing:

In [ ]:
ib.show_graph(p)